# Chapter 10: Observational Studies vs. Designed Experiments

## Lesson Title: "Crash Course: Types of Studies in Engineering"

## Learning Goals
1.  **Observational Studies**: Differentiate between **Retrospective** (looking back) and **Prospective** (looking forward) studies.
2.  **Lurking Variables**: Identify hidden variables that cause misleading correlations (e.g., Ice Cream vs. Drowning).
3.  **Designed Experiments**: Apply the **Four Principles**: Control, Randomize, Replicate, and Block.
4.  **Vocabulary**: Define Factors, Levels, Treatments, Blinding, Placebos, and Statistical Significance.

## Part 1: Observational Studies
An Observational Study is valuable for discovering trends but **cannot prove cause-and-effect**. Researchers simply observe choices; they do not assign them.

### Retrospective vs. Prospective
*   **Retrospective**: Subjects are selected and their **previous conditions** or behaviors are determined. (e.g., Analyzing 10 years of past bridge maintenance logs).
*   **Prospective**: Subjects are followed to observe **future outcomes**. (e.g., Identifying 100 new bridges and tracking their cracks for the next 10 years).

### Exercise: Classify the Study
For each of the following scenarios, decide if it is **Retrospective** or **Prospective**.

1.  **Screen Time**: A university recruits 200 students and asks them to log their daily screen time and sleep duration for the next 6 months.
2.  **Study Habits**: A researcher investigates how study habits impact test scores. They track a group of students over the course of a school year, recording their study habits and test scores at regular intervals.
3.  **Smoking**: A group of researchers wanted to study the effects of smoking on lung health. They gathered medical records of 500 people who had already been diagnosed with lung cancer and examined their smoking histories to determine if there was a pattern.
4.  **Pesticides**: A team of scientists studies the potential connection between pesticide exposure and Parkinson’s disease. They identify a group of people who have been diagnosed with Parkinson’s and review their work histories to see if they were exposed to pesticides in the past.
5.  **Sugary Beverages**: Researchers are interested in the long-term health effects of drinking sugary beverages. They recruit 1,000 participants who currently do not have diabetes and track their beverage consumption and health outcomes over the next 10 years.
6.  **Neighborhood Cohesion**: A sociologist examines the effect of community events on neighborhood cohesion. They review past community event attendance records and survey long-time residents about their sense of neighborhood belonging.
7.  **Tree Planting**: An environmental scientist studies the impact of tree planting on reducing local temperatures. They identify neighborhoods planning large tree-planting initiatives and measure the area's temperature changes over the next decade.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# ------------------------------------------------------------------------
# ANSWER KEY WIDGET
# ------------------------------------------------------------------------

btn_reveal = widgets.Button(description="Reveal Answers")
output_reveal = widgets.Output()

def show_answers(b):
    with output_reveal:
        clear_output()
        print("--- ANSWER KEY ---")
        print("1. Prospective (Tracking for next 6 months)")
        print("2. Prospective (Tracking over school year)")
        print("3. Retrospective (Examining past records)")
        print("4. Retrospective (Reviewing work histories)")
        print("5. Prospective (Tracking over next 10 years)")
        print("6. Retrospective (Reviewing past records)")
        print("7. Prospective (Measuring changes over next decade)")

btn_reveal.on_click(show_answers)
display(widgets.VBox([widgets.Label("Check your work above, then click to see if you were right:"), btn_reveal, output_reveal]))

### Lurking Variables
A **Lurking Variable** is a hidden cause that links two things that aren't actually related. 
*   It is usually thought of as a **prior cause** of both X and Y.

**Classic Example**: "Neighborhoods with more **Ice Cream Shops** have higher rates of **Drowning**."
*   Does ice cream cause drowning?
*   **NO**. The Lurking Variable is **Summer Heat**. Hot weather makes people eat ice cream AND swim more.

Let's look at a manufacturing example below.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------
# DEMO: LURKING VARIABLE IN CAR ACCIDENTS
# ------------------------------------------------------------------------

def generate_crash_data(n=300):
    np.random.seed(42)
    
    # Lurking Variable: Driver Aggression Level (Hidden from the dataset!)
    # 0 = Calm, 1 = Aggressive
    aggression = np.random.choice([0, 1], size=n, p=[0.7, 0.3])
    
    data = []
    for i in range(n):
        is_aggressive = aggression[i]
        
        # Aggressive drivers prefer Red Sports Cars
        if is_aggressive:
            car_color = np.random.choice(['Red', 'Black'], p=[0.8, 0.2])
            speed = np.random.normal(90, 10) # Fast
        else:
            car_color = np.random.choice(['White', 'Silver', 'Blue', 'Red'], p=[0.3, 0.3, 0.3, 0.1])
            speed = np.random.normal(60, 5) # Normal
            
        # Accident risk depends on SPEED, not COLOR
        accident_risk = (speed - 50) / 100 
        has_accident = 1 if np.random.random() < accident_risk else 0
        
        data.append({'Color': car_color, 'Accident': has_accident})
        
    return pd.DataFrame(data)

df_cars = generate_crash_data()

plt.figure(figsize=(8, 4))
sns.barplot(data=df_cars, x='Color', y='Accident', ci=None, palette='viridis')
plt.title("Observational Data: Accident Rate by Car Color")
plt.ylabel("Probability of Accident")
plt.show()

print("Does Red Paint cause accidents? Or is it the driver?")

## Part 2: Designed Experiments

To prove cause-and-effect, we must **Design an Experiment**. The experimenter deliberately manipulates factors to create treatments.

### Vocabulary
*   **Experimental Units**: The things we test on (e.g., The Cars). When humans are involved, they are **Subjects**.
*   **Factors**: The variables we manipulate (e.g., Brake Type).
*   **Levels**: The specific values of the factor (e.g., Brake System A vs Brake System B).
*   **Treatments**: A combination of specific levels from all factors.

### The Four Principles
1.  **Control**: We control sources of variation (other than the factors) to make conditions similar for all groups.
2.  **Randomize**: We assign treatments randomly to equalize unknown sources of variation. It doesn't eliminate effects, but **spreads them out**.
3.  **Replicate**: Repeat the experiment on many subjects. **"An anecdote (one subject) is not data."**
4.  **Block**: Group similar individuals together (e.g., by Road Condition) and randomize within the blocks. This removes variability coming from the blocking variable.

### Diagramming the Experiment
Below is a "Tree Diagram" of our Braking Experiment. We start with the Population, Block by Road Surface, and then Randomize Assignment.

In [ ]:
# Code to just print a text-based diagram of the experiment design
def print_diagram():
    print("           [ All 20 Cars ] ")
    print("                  | ")
    print("         BLOCKING INSTRUCTION ")
    print("      (Group by Road Condition) ")
    print("          /               \ ")
    print("    [10 Dry Road]     [10 Wet Road] ")
    print("       /     \           /     \ ")
    print("  RANDOMIZE  RANDOMIZE  RANDOMIZE  RANDOMIZE")
    print("    /           \       /           \ ")
    print(" [sys A]     [Sys B] [Sys A]     [Sys B]")
    print("    |           |       |           | ")
    print(" COMPARE     COMPARE  COMPARE     COMPARE")
    print(" RESULTS     RESULTS  RESULTS     RESULTS")

print_diagram()

### Blinding and Placebos
*   **Blinding**: Hiding the treatment info to prevent bias from those who *influence* result (drivers) or *evaluate* results (judges).
*   **Single-Blind**: Everyone in one class (e.g., drivers) is blinded.
*   **Double-Blind**: Everyone in BOTH classes (drivers and judges) is blinded.
*   **Placebo**: A "fake" treatment (like a sugar pill) so subjects don't know if they are getting the real thing. In engineering, this is often the "Business as Usual" or Control standard.

--- 

### Statistically Significant
How large do the differences need to be to say there is a real difference?
*   **Definition**: Differences that are larger than we’d get just from **randomization alone** are called **Statistically Significant**.
*

In [ ]:
# ------------------------------------------------------------------------
# SIMULATION: THE BRAKING EXPERIMENT (With Blocking & Randomization)
# ------------------------------------------------------------------------

style = {'description_width': 'initial'}

randomize_check = widgets.Checkbox(
    value=False, 
    description='ENABLE Randomized Assignment',
    style=style
)

output_exp = widgets.Output()

def run_brake_experiment(b):
    is_randomized = randomize_check.value
    
    with output_exp:
        clear_output(wait=True)
        
        # Setup the "environment"
        # 20 Test Runs. First 10 are Dry, Last 10 are Wet (Nature decides this)
        conditions = ['Dry'] * 10 + ['Wet'] * 10
        
        # ASSIGN TREATMENTS (Brake System A vs B)
        if not is_randomized:
            # BAD SCIENCE: Lazy engineer tests A in the morning (Dry) and B in the afternoon (Wet)
            treatments = ['System A'] * 10 + ['System B'] * 10
            design_score = "POOR (Confounded)"
        else:
            # GOOD SCIENCE: Randomly assign A or B to each run
            treatments = np.random.choice(['System A', 'System B'], size=20, p=[0.5, 0.5])
            design_score = "EXCELLENT (Randomized)"
            
        # SIMULATE RESULTS (Stopping Distance in meters)
        # The Loop runs 20 times. This is the REPLICATION Principle.
        results = []
        for i in range(20):
            base_dist = 40 # avg stopping distance
            
            # Treatment Effect
            if treatments[i] == 'System A': dist = base_dist - 5
            else: dist = base_dist + 0
                
            # Blocking Factor Effect
            if conditions[i] == 'Wet': dist += 20 
            
            # Random noise
            dist += np.random.normal(0, 2)
            
            results.append({
                'Run_ID': i+1,
                'Road_Condition': conditions[i],
                'Brake_System': treatments[i],
                'Stop_Distance_m': dist
            })
            
        df_exp = pd.DataFrame(results)
        
        # VISUALIZE
        plt.figure(figsize=(10,6))
        sns.boxplot(data=df_exp, x='Brake_System', y='Stop_Distance_m', hue='Road_Condition')
        plt.title(f"Experiment Results (Design: {design_score})")
        plt.ylabel("Stopping Distance (meters) [Lower is Better]")
        plt.show()
        
        # ANALYSIS TEXT
        if not is_randomized:
            print("\n⚠ CONFOUNDING ALERT: System A looks amazing, but it was only tested on DRY roads!")
            print("We cannot separate the effect of the Brakes from the effect of the Water.")
        else:
            print("\n✅ SUCCESS: Randomization ensured both systems were tested on Wet and Dry roads.")
            print("The Boxplot shows System A is slightly better in BOTH conditions.")

btn = widgets.Button(description="Run Experiment")
btn.on_click(run_brake_experiment)

display(widgets.HBox([randomize_check, btn]))
display(output_exp)

## Discussion Questions

1.  **Ethics**: Why is it unethical to run a "Designed Experiment" on the effects of Smoking? (Hint: Can we force 500 people to smoke?)
2.  **Blinding**: If the driver *knew* he was testing the "New Experimental Brakes", how might that bias the results? What kind of Blinding would fix this?
3.  **Lurking Variables**: A study finds that students who have tutors get *lower* grades than students who don't. Does tutoring make you dumber? What is the lurking variable (the missing reason)?
4.  **Confounding**: In Experiment 1 (above), can you separate the effect of the Brakes from the effect of the Rain? Why is this called "Confounded"?